In [1]:
import time
import random
import os
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from numpy import genfromtxt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import glob
import timeit
import csv 
import math

In [8]:
# parent_dir =  '/data/hoan/amromics/kover2_paper/data/SaPatric/prokkaMore/'
parent_dir =  '/data/hoan/amromics/kover2_paper/data/KpPatric/prokkaMore/'
data_dir = glob.glob(parent_dir + '*.fna')

In [9]:
n_samples = len(data_dir)
n_folds = 4 # divide the data into 4 folds

In [10]:
def get_fold(idx):
    return str(math.floor(n_folds*idx/(n_samples+1)))

In [25]:
fold_idx = [0]
for idx in range(len(data_dir)):
    if get_fold(idx) < get_fold(idx+1):
        fold_idx.append(idx+1)
fold_idx.append(n_samples)

In [26]:
fold_idx

[0, 583, 1167, 1751, 2334]

In [ ]:
for idx in range(len(data_dir)):
    to_dir = parent_dir + 'Fold' + get_fold(idx)
    cmd = 'cp '+data_dir[idx]+' ' + to_dir
    if not os.path.isdir(to_dir):
        os.system('mkdir ' + to_dir)
    # print(to_dir)
    os.system(cmd)

# First run kounta

In [ ]:
# conda activate amromics
# kounta --kmer 31 --out /data/hoan/amromics/prediction/data/Ecoli1936_DNAKmerk31Fold1.tsv /data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/Fold1/*.fna
# kounta --kmer 31 --out /data/hoan/amromics/prediction/data/Ecoli1936_DNAKmerk31Fold2.tsv /data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/Fold2/*.fna
# kounta --kmer 31 --out /data/hoan/amromics/prediction/data/Ecoli1936_DNAKmerk31Fold3.tsv /data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/Fold3/*.fna
# kounta --kmer 31 --out /data/hoan/amromics/prediction/data/Ecoli1936_DNAKmerk31Fold4.tsv /data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/Fold4/*.fna

In [ ]:
# kounta --kmer 31 --out /data/hoan/amromics/prediction/data/SaPatric_DNAKmerk31Fold1.tsv /data/hoan/amromics/kover2_paper/data/SaPatric/prokkaMore/Fold1/*.fna
# kounta --kmer 31 --out /data/hoan/amromics/prediction/data/SaPatric_DNAKmerk31Fold2.tsv /data/hoan/amromics/kover2_paper/data/SaPatric/prokkaMore/Fold2/*.fna
# kounta --kmer 31 --out /data/hoan/amromics/prediction/data/SaPatric_DNAKmerk31Fold3.tsv /data/hoan/amromics/kover2_paper/data/SaPatric/prokkaMore/Fold3/*.fna

In [2]:
data_name = 'KpPatric' # 'Ecoli1936'

In [ ]:
print('====================================== Run Kounta ==================================================')
print('====================================== ********** ==================================================')

In [13]:
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate amromics && '
kounta_bin = 'kounta --kmer 31 --tempdir /data/hoan/tmpdir --out '
dataoutDir = '/data/hoan/amromics/prediction/data/'+data_name+'_DNAKmerk31Fold'
datainDir = parent_dir + 'Fold'
for i in range(n_folds):
    cmline = conda_dir + kounta_bin + dataoutDir + str(i)+'.tsv ' + datainDir + str(i) + '/*.fna'
    print(cmline)
    os.system(cmline)
    os.system('rm -r /data/hoan/tmpdir/*')

source ~/miniconda3/etc/profile.d/conda.sh && conda activate amromics && kounta --kmer 31 --out /data/hoan/amromics/prediction/data/SaPatric_DNAKmerk31Fold0.tsv /data/hoan/amromics/kover2_paper/data/KpPatric/prokkaMore/Fold0/*.fna
source ~/miniconda3/etc/profile.d/conda.sh && conda activate amromics && kounta --kmer 31 --out /data/hoan/amromics/prediction/data/SaPatric_DNAKmerk31Fold1.tsv /data/hoan/amromics/kover2_paper/data/KpPatric/prokkaMore/Fold1/*.fna
source ~/miniconda3/etc/profile.d/conda.sh && conda activate amromics && kounta --kmer 31 --out /data/hoan/amromics/prediction/data/SaPatric_DNAKmerk31Fold2.tsv /data/hoan/amromics/kover2_paper/data/KpPatric/prokkaMore/Fold2/*.fna


In [ ]:
# metadata_panta = pd.read_csv("/data/hoan/amromics/prediction/data/Ecoli1936metafiles/metadata_final.csv")
# metadata_panta = pd.read_csv("/data/hoan/amromics/prediction/data/Kpmetadata_final.csv")
# metadata_panta.head()

# Process DNA data

## Feature selection phase I: select features with high variance

In [ ]:
print('====================================== Run Phase I FS ==================================================')
print('====================================== ********** ==================================================')

In [ ]:
for idx in range(n_folds):
    aafold = str(idx)
    full_data = []
    full_name = []
    count = 0;
    print('...')
    print("aafold: ", aafold)
    with open('/data/hoan/amromics/prediction/data/'+data_name+'_DNAKmerk31Fold'+aafold+'.tsv', 'r') as csvfile: 
        students_reader = csv.reader(csvfile, delimiter='\t') 
        for row in students_reader: 
            data_row = np.array(row[1:])
            if count > 0:
                data_row = data_row.astype(np.int)
                if (np.var(data_row)) > 0.05:
                    full_data.append(data_row)
                    full_name.append(row[0])
            count += 1
            # if count > 100:
            #     break;

    full_data = np.array(full_data)
    feature = np.array(full_name)
    # should use VT5
    np.save('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold'+aafold+'_DNAmat_VT1.npy', full_data) # save numpy array
    np.save('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold'+aafold+'_DNAmat_VT1_features.npy', feature) # save numpy array

In [ ]:
# stop

In [ ]:
print('====================================== Run Phase II: FS ==================================================')
print('====================================== ********** ==================================================')

## Feature selection Phase II: select features which correlate with labels

In [ ]:
from pangraph.utils import binary_label
from sklearn.feature_selection import mutual_info_classif, chi2

In [9]:
# metadata_panta = pd.read_csv("/data/hoan/amromics/prediction/data/Sametadata_final.csv")
metadata_panta = pd.read_csv("/data/hoan/amromics/prediction/data/Kpmetadata_final.csv")

In [ ]:
aafold = '1' # '1', '2', '3', '4'

In [ ]:
for foldidx in range(n_folds):
    aafold = str(foldidx)
    print('aafold: ', aafold)
    kmer_matrix_VT1_fi = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold'+aafold+'_DNAmat_VT1.npy')
    selected_features_fi = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold'+aafold+'_DNAmat_VT1_features.npy')
    kmer_matrix_VT1_fi = kmer_matrix_VT1_fi.T

    # if aafold == '1':
    #     start_idx = 0; end_idx = 600
    # elif aafold == '2':
    #     start_idx = 600; end_idx = 1200
    # else:
    #     start_idx = 1200; end_idx = metadata_panta.shape[0]
    start_idx = fold_idx[foldidx]
    end_idx = fold_idx[foldidx+1]

    mutual_mat = []
    for idx in range(2, metadata_panta.shape[1]):
        # y_class = metadata_panta.iloc[:600,idx].values
        y_class = metadata_panta.iloc[start_idx:end_idx, idx].values
        print(metadata_panta.columns[idx])
        y, nonenan_index = binary_label(y_class) # v6
        if len(nonenan_index) > 10:
            pa_matrix_new = kmer_matrix_VT1_fi[nonenan_index, ]
            y_new = y[nonenan_index].astype(int)
        # if len(y_new) > 10:
            scores, pvalue = chi2(pa_matrix_new, y_new)
            mutual_mat.append(scores)
    mutual_mat = np.array(mutual_mat)
    mutual_mat_mean = mutual_mat.mean(axis=0)
    top_features = np.argsort(mutual_mat_mean)[::-1][:100000]
    kmer_matrix_VT_top_features = kmer_matrix_VT1_fi[:,top_features]
    selected_features_top = selected_features_fi[top_features]
    np.save('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold'+aafold+'_DNAmat_VT1_top_features.npy', kmer_matrix_VT_top_features) # save numpy array
    np.save('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold'+aafold+'_DNAmat_VT1_features_top_features.npy', selected_features_top) # save numpy array

In [ ]:
# stop

In [ ]:
print('====================================== Concat and save ==================================================')
print('====================================== ********** ==================================================')
print("Do it manually")

## Concat 3 datasets together (aafold for 0, 1, 2 and ...)

In [ ]:
# data1 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold1_DNAmat_VT1_top_features.npy') # save numpy array
# feature1 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold1_DNAmat_VT1_features_top_features.npy') # save numpy array
# df1 = pd.DataFrame(data = data1, columns = feature1)
# #
# data2 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold2_DNAmat_VT1_top_features.npy') # save numpy array
# feature2 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold2_DNAmat_VT1_features_top_features.npy') # save numpy array
# df2 = pd.DataFrame(data = data2, columns = feature2)
# #
# data3 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold3_DNAmat_VT1_top_features.npy') # save numpy array
# feature3 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold3_DNAmat_VT1_features_top_features.npy') # save numpy array
# df3 = pd.DataFrame(data = data3, columns = feature3)
#
# data4 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold4_DNAmat_VT1_top_features.npy') # save numpy array
# feature4 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold4_DNAmat_VT1_features_top_features.npy') # save numpy array
# df4 = pd.DataFrame(data = data4, columns = feature4)

In [3]:
data0 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold0_DNAmat_VT1_top_features.npy') # save numpy array
feature0 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold0_DNAmat_VT1_features_top_features.npy') # save numpy array
df0 = pd.DataFrame(data = data0, columns = feature0)
#
data1 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold1_DNAmat_VT1_top_features.npy') # save numpy array
feature1 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold1_DNAmat_VT1_features_top_features.npy') # save numpy array
df1 = pd.DataFrame(data = data1, columns = feature1)
#
data2 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold2_DNAmat_VT1_top_features.npy') # save numpy array
feature2 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold2_DNAmat_VT1_features_top_features.npy') # save numpy array
df2 = pd.DataFrame(data = data2, columns = feature2)
#
data3 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold3_DNAmat_VT1_top_features.npy') # save numpy array
feature3 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold3_DNAmat_VT1_features_top_features.npy') # save numpy array
df3 = pd.DataFrame(data = data3, columns = feature3)
#
data4 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold4_DNAmat_VT1_top_features.npy') # save numpy array
feature4 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold4_DNAmat_VT1_features_top_features.npy') # save numpy array
df4 = pd.DataFrame(data = data4, columns = feature4)
#
data5 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold5_DNAmat_VT1_top_features.npy') # save numpy array
feature5 = np.load('/data/hoan/amromics/prediction/data/'+data_name+'kmer_Fold5_DNAmat_VT1_features_top_features.npy') # save numpy array
df5 = pd.DataFrame(data = data5, columns = feature5)

In [ ]:
# df1.shape, df2.shape, df3.shape, df4.shape

In [4]:
# df_full = pd.concat([df1,df2,df3, df4], axis=0, ignore_index=True) 
# df_full = pd.concat([df1,df2,df3], axis=0, ignore_index=True) 
df_full = pd.concat([df0, df1,df2,df3, df4, df5], axis=0, ignore_index=True) 

In [5]:
df_full = df_full.fillna(0)

In [6]:
snp_mat = df_full.values

In [7]:
snp_mat.shape

(2334, 207196)

In [8]:
np.save('/data/hoan/amromics/prediction/data/'+data_name+'kmer_FULLMAT_VT1_DNA.npy', snp_mat) # save numpy array

In [10]:
metadata_panta.shape

(2334, 26)